In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the data
corpus_url = '/content/drive/MyDrive/kalapa/MEDICAL/corpus.zip'

In [ ]:
!mkdir data
!unzip /content/drive/MyDrive/kalapa/MEDICAL/corpus.zip -d /content/data

Archive:  /content/drive/MyDrive/kalapa/MEDICAL/corpus.zip
   creating: /content/data/corpus/
  inflating: /content/data/__MACOSX/._corpus  
  inflating: /content/data/corpus/liet-mat  
  inflating: /content/data/__MACOSX/corpus/._liet-mat  
  inflating: /content/data/corpus/dieu-tri-chung-roi-loan-tien-man-kinh-luu-y-ve-an-uong  
  inflating: /content/data/__MACOSX/corpus/._dieu-tri-chung-roi-loan-tien-man-kinh-luu-y-ve-an-uong  
  inflating: /content/data/corpus/suy-tim-mat-bu  
  inflating: /content/data/__MACOSX/corpus/._suy-tim-mat-bu  
  inflating: /content/data/corpus/suy-gian-tinh-mach-sau-chi-duoi  
  inflating: /content/data/__MACOSX/corpus/._suy-gian-tinh-mach-sau-chi-duoi  
  inflating: /content/data/corpus/xuat-huyet-da-day  
  inflating: /content/data/__MACOSX/corpus/._xuat-huyet-da-day  
  inflating: /content/data/corpus/tieu-duong-tuyp-1  
  inflating: /content/data/__MACOSX/corpus/._tieu-duong-tuyp-1  
  inflating: /content/data/corpus/viem-gan-man-tinh  
  inflating: 

In [ ]:
import glob
import os
files = glob.glob(os.path.join('/content/data/corpus/', "*"), recursive=True)

In [ ]:
from tqdm import tqdm

In [ ]:
dataset = []
for file in tqdm(files):
  f = open(file, "r")
  dataset.append(f.read())

100%|██████████| 603/603 [00:00<00:00, 8755.19it/s]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
dataset = []

def count_words(phrase):
  return len(phrase.split(' '))

splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=50,
    add_start_index=False,
    length_function=count_words
)

for file in tqdm(files):
  f = open(file, "r")
  raw = f.read()
  for chunk in splitter.split_text(raw):
    dataset.append(chunk)

100%|██████████| 603/603 [00:00<00:00, 784.79it/s]


In [ ]:
def clean(dataset):
  res = []
  for corpus in tqdm(dataset):
    finder = "CHUYÊN MỤC BỆNH HỌC > "
    x = corpus.find(finder)
    res.append(corpus[x + len(finder):])
  return res


In [ ]:
dataset = clean(dataset)

100%|██████████| 1741/1741 [00:00<00:00, 142636.65it/s]


In [ ]:
train_id = len(dataset) - 10

In [ ]:
import datasets
train_dict = {'text': dataset[:train_id]}
test_dict = {'text': dataset[train_id:]}
train_ds = datasets.Dataset.from_dict(train_dict)
test_ds = datasets.Dataset.from_dict(test_dict)

In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], max_length=256, truncation=True, return_attention_mask=True, padding="max_length")

In [ ]:
tokenized_train = train_ds.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/1731 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = test_ds.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
block_size = tokenizer.model_max_length

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
train_datasets = tokenized_train.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/1731 [00:00<?, ? examples/s]

In [ ]:
test_datasets = tokenized_test.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
model_name = "vinai/phobert-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name, is_decoder=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasets,
    eval_dataset=test_datasets,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.472965
2,No log,2.083993
3,3.081700,1.996971


TrainOutput(global_step=651, training_loss=2.841055968206965, metrics={'train_runtime': 411.5752, 'train_samples_per_second': 12.617, 'train_steps_per_second': 1.582, 'total_flos': 683678339937792.0, 'train_loss': 2.841055968206965, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 7.37


In [ ]:
!mkdir /content/model2

In [ ]:
trainer.save_model("/content/model2")

In [ ]:
question_url = '/content/drive/MyDrive/kalapa/MEDICAL/public_test.csv'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(question_url)

In [ ]:
df['question'][0]

'Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?'

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch
import numpy as np

pretrained_model = "/content/model2"
model = AutoModelForCausalLM.from_pretrained(pretrained_model, is_decoder=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def calculate_dist(a, b):
  if (a.shape[0] != b.shape[0]):
    zero = np.zeros((b.shape[0] - a.shape[0], a.shape[1]))
    c = np.concatenate([a, zero], axis=0)
  print(np.linalg.norm(b - c))

In [ ]:
res = []
for idx in tqdm(range(100)):
  feature = []

  question = df['question'][idx]
  input_ids = torch.tensor([tokenizer.encode(question, max_length=256, truncation=True, return_attention_mask=True, padding="max_length")])
  with torch.no_grad():
      features = model(input_ids)
      question_features = features.logits.squeeze()
      # print(question_features.shape)

  for option in range(6):
    option = df[f'option_{option + 1}'][idx]
    try:
      temp = option.find('.')
      option = option[temp + 1:]
    except:
      break
    input_ids = torch.tensor([tokenizer.encode(question,
                                              option,
                                              max_length=256,
                                              truncation=True,
                                              return_attention_mask=True,
                                              padding="max_length")])
    with torch.no_grad():
        features = model(input_ids)
        option_features = features.logits.squeeze()
    feature.append(np.linalg.norm(question_features - option_features))
  res.append(np.argmax(feature))

100%|██████████| 100/100 [10:15<00:00,  6.15s/it]


In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
import csv

with open('submission.csv', 'w+', newline='') as file:
  writer = csv.writer(file)

  field = ["id", "answer"]
  writer.writerow(field)
  for i, data in enumerate(res):
    id = 3
    while isinstance(df[f'option_{id}'][i], str):
      id += 1
    id -= 1
    temp = to_categorical(data, num_classes=id)
    s = int(0)
    for each in temp:
      s = s*10 + each
    print(int(s), data, id)
    writer.writerow([df['id'][i], int(s)])
    # writer.writerow(field)

1000 0 4
1000 0 4
10 2 4
1 1 2
1 3 4
10 2 4
100 1 4
1 1 2
1 2 3
1 1 2
1 4 5
1 2 3
100 0 3
1 2 3
10 1 3
10 1 3
100 1 4
1000 0 4
1 2 3
1 2 3
1 2 3
10 1 3
100 0 3
10 1 3
1000 0 4
1 1 2
10 1 3
1 2 3
100 1 4
100 1 4
100 1 4
100 1 4
10 1 3
1 2 3
1 2 3
100 0 3
10 2 4
10 0 2
1 2 3
10 0 2
1 2 3
10 1 3
100 0 3
100 1 4
1 4 5
100 0 3
1 2 3
1 1 2
10 1 3
1 2 3
10 0 2
100 0 3
1 2 3
10 1 3
1 2 3
10 2 4
10 0 2
100 0 3
100 0 3
100 0 3
1 2 3
10 2 4
1 2 3
1000 0 4
100 0 3
100 0 3
1 2 3
10 1 3
1000 0 4
1000 0 4
1 2 3
1 2 3
1 2 3
100 1 4
10 1 3
1 2 3
100 1 4
100 1 4
100 1 4
1 2 3
10000 0 5
100 0 3
10 2 4
100 1 4
10 1 3
1 4 5
10 1 3
10 2 4
1 2 3
1 2 3
1000 0 4
100 1 4
100 1 4
1000 0 4
10 2 4
1000 0 4
10 2 4
1 3 4
1 3 4
1 2 3
